# How do I validate a single field?

## First Way: Write `validate_<Field Name>`

If the field name is X, then we can write a method called `validate_X` with the following signature:

    def validate_X(self, json_obj, value_of_x)

In the above signature, `json_obj` is the object passed into the constructor to create the object. In case we need some other information from that structure. `value_of_x` is the value of the field being validated. Here is an example:

In [35]:
import string

import schematics


VALID_CHARS = set(string.ascii_letters + string.digits + "_")


class UserMetadata(schematics.Model):
    name = schematics.types.StringType(required=True)
    display_name = schematics.types.StringType(serialized_name="displayName")
    description = schematics.types.StringType()
    
    def validate_name(self, json_obj, name):
        invalid_chars = "".join(set(name) - VALID_CHARS)
        if invalid_chars:
            raise schematics.exceptions.ValidationError(
                f"Expect only letters, digits and underscore"
                f", but found {invalid_chars!r}"
                f" in {name}"
            )

def verify_user_metadata(json_obj):
    user_metadata = UserMetadata(json_obj)
    try:
        user_metadata.validate()
    except schematics.exceptions.DataError as error:
        print(error)
    else:
        print("Valid")

### Try it out

In [13]:
# This should be valid
verify_user_metadata(dict(
    name="user_1",
))

Valid


In [16]:
# Invalid chars found
verify_user_metadata(dict(
    name="user-1#"
))

{"name": ["Expect only letters, digits and underscore, but found '-#' in user-1#"]}


### What Is the Use of `json_obj`?

Sometimes, we want to validate a field against another field, this is where the `json_obj` comes in. In the following example, we create a `User` object and if the user is root, then that user must be an admin.

In [44]:
import schematics

class User(schematics.Model):
    alias = schematics.types.StringType(required=True)
    is_admin = schematics.types.BooleanType(default=False)
    
    def validate_is_admin(self, json_obj, is_admin_value):
        if json_obj["alias"] == "root" and is_admin_value is not True:
            raise schematics.exceptions.ValidationError(
                "root must be an admin."
            )
            
            
def verify_user(json_obj):
    user = User(json_obj)
    try:
        user.validate()
    except schematics.exceptions.DataError as error:
        print(f"Error: {error}")
    else:
        print("Valid")

In [45]:
# Non-root
verify_user(dict(alias="johan", is_admin=True))

Valid


In [46]:
# Root, is admin
verify_user(dict(alias="root", is_admin=True))

Valid


In [47]:
# Invalid
verify_user(dict(alias="root"))

Error: {"is_admin": ["root must be an admin."]}


# Second Way: Write a Validation Function

If we found having to perform the same validation for different fields, we can write a validation function and use that:

In [23]:
import re

import schematics


def validate_simple_us_phone(value):
    """
    Validate a phone number.
    
    For the sake of simplicity, we only validate NNN-NNN-NNNN.
    """
    valid_pattern = re.compile(r"\d{3}-\d{3}-\d{4}")
    if not valid_pattern.match(value):
        raise schematics.exceptions.ValidationError(f"Invalid phone: {value}")


class User(schematics.Model):
    name = schematics.types.StringType(required=True)
    mobile_phone = schematics.types.StringType(validators=[validate_simple_us_phone])
    work_phone = schematics.types.StringType(validators=[validate_simple_us_phone])
    
    
def verify_user(json_obj):
    user = User(json_obj)
    try:
        user.validate()
    except schematics.exceptions.DataError as error:
        print(error)
    else:
        print("valid")

In [24]:
# Valid
verify_user(dict(name="Johan", mobile_phone="425-772-3359"))

valid


In [25]:
# Bad mobile phone number
verify_user(dict(name="Johan", mobile_phone="4257723359"))

{"mobile_phone": ["Invalid phone: 4257723359"]}


In [26]:
# Bad phone numbers
verify_user(dict(name="Johan", mobile_phone="4257723359", work_phone="425-7742"))

{"mobile_phone": ["Invalid phone: 4257723359"], "work_phone": ["Invalid phone: 425-7742"]}


## Third Way: Create Custom Type

If we found we repeatedly need the same validation, another approach is to write a custom type. In this case, we will write a `SimpleUSPhoneType`. Within this type, we can write a number of `validate_*` functions to validate the value.

In [32]:
import re
import string

import schematics


class SimpleUSPhoneType(schematics.types.StringType):
    """
    For simplicity, we only allow NNN-NNN-NNNN format.
    """
    def validate_first_digit(self, value):
        """The first digit cannot be a 1."""
        if value[0] == "1":
            raise schematics.exceptions.ValidationError(
                f"First digit must not be 1: {value}"
            )
            
    def validate_format(self, value):
        valid_pattern = re.compile(r"\d{3}-\d{3}-\d{4}")
        if not valid_pattern.match(value):
            raise schematics.exceptions.ValidationError(
                f"Invalid format: {value}"
            )
        


class User(schematics.Model):
    name = schematics.types.StringType(required=True)
    mobile_phone = SimpleUSPhoneType()
    work_phone = SimpleUSPhoneType()
    


def verify_user(json_obj):
    user = User(json_obj)
    try:
        user.validate()
    except schematics.exceptions.DataError as error:
        print(error)
    else:
        print("valid")

In [33]:
# Valid
verify_user(dict(name="Johan", mobile_phone="345-678-9012", work_phone="555-123-4567"))

valid


In [34]:
# Invalid phone numbers
verify_user(dict(name="Johan", mobile_phone="3456789012", work_phone="155-123-4567"))

{"mobile_phone": ["Invalid format: 3456789012"], "work_phone": ["First digit must not be 1: 155-123-4567"]}
